First we will pull some data to work with. The specifics of this data pull have been anonymized.

In [1]:
import sys
import pandas as pd
import numpy as np
import pymysql.cursors
import argparse
import datetime 
import configparser
import itertools

config= configparser.ConfigParser()
config.read('db.ini')
dbhost = config.get('database','dbhost')
username = config.get('database','username')
port = config.get('database','port')
password = config.get('database','password')
dbname = config.get('database','dbname')
ssl_cert = config.get('database','ssl_cert')

#print(dbname)
conn= pymysql.connect(host= dbhost,
                                 port= int(port),
                                 user= username,
                                 password= password,
                                 db= dbname ,
                                 charset='utf8mb4',
                                 ssl={'ssl':{'ca' : ssl_cert }},
                                 cursorclass=pymysql.cursors.DictCursor)

cur = conn.cursor()
query1 = (
"""
SELECT a.*
, a.`total_spend`/a.`total_clicks` as `CPC`
, a.`total_clicks`/a.`total_impressions` as `CTR`
, a.`total_orders`/a.`total_impressions` as `CR_impressions`
, a.`total_orders`/a.`total_clicks` as `CR_clicks`
, a.`total_ad_sales`/a.`total_spend` as `ROAS`
, a.`total_sales`/a.`total_orders` as `AOV`
, a.`new_to_brand_customers`/a.`total_spend` as `CAC`


from
(select`brandid`,
`pst_date`,
`minWeekDate`,
`total_units`,
`total_sales`,
`total_orders`,
`page_views`,
`sessions`,
`sp_impressions`+`sb_impressions`+`sd_impressions`+`sbv_impressions` as `total_impressions`,
`sp_clicks`+`sb_clicks`+`sd_clicks`+`sbv_clicks` as `total_clicks`,
`sp_spend`+`sb_spend`+`sd_spend`+`sbv_spend` as `total_spend`,
`sp_sales`+`sb_sales`+`sd_sales`+`sbv_sales` as `total_ad_sales`,
`new_to_brand_customers`
FROM `new_brand_metrics` WHERE `brandid` = '143'
or `brandid` = '113'
or `brandid` = '46'
or `brandid` = '114'
or `brandid` = '55'
or `brandid` = '27'
or `brandid` = '2'
or `brandid` = '50'
or `brandid` = '11'
) a
""")

cur.execute(query1)

df = pd.DataFrame(data=cur.fetchall())
cur.close()
conn.close()

df.head()

,brandid,pst_date,minWeekDate,total_units,total_sales,total_orders,page_views,sessions,total_impressions,total_clicks,total_spend,total_ad_sales,new_to_brand_customers,CPC,CTR,CR_impressions,CR_clicks,ROAS,AOV,CAC
0,55,2014-07-16,2014-07-13,0,0.00,0,0,0,0,0,0.000,0.000,0,None,None,None,None,None,None,None
1,55,2014-07-17,2014-07-13,0,0.00,0,0,0,0,0,0.000,0.000,0,None,None,None,None,None,None,None
2,55,2014-07-18,2014-07-13,0,0.00,0,0,0,0,0,0.000,0.000,0,None,None,None,None,None,None,None
3,55,2014-08-06,2014-08-03,0,0.00,0,0,0,0,0,0.000,0.000,0,None,None,None,None,None,None,None
4,55,2014-08-07,2014-08-03,0,0.00,0,0,0,0,0,0.000,0.000,0,None,None,None,None,None,None,None


# Brand Latest Month Metrics

Here we will pull the metrics for the brand for the latest month in the data set.

In [2]:
brand_id = 143
latest_month = '2022-01'
agg_year = '2021'

In [3]:
df['month'] = df['pst_date'].astype('string')
df['month'] = df['month'].str.slice(0, 7)
df.head()

,brandid,pst_date,minWeekDate,total_units,total_sales,total_orders,page_views,sessions,total_impressions,total_clicks,...,total_ad_sales,new_to_brand_customers,CPC,CTR,CR_impressions,CR_clicks,ROAS,AOV,CAC,month
0,55,2014-07-16,2014-07-13,0,0.00,0,0,0,0,0,...,0.000,0,None,None,None,None,None,None,None,2014-07
1,55,2014-07-17,2014-07-13,0,0.00,0,0,0,0,0,...,0.000,0,None,None,None,None,None,None,None,2014-07
2,55,2014-07-18,2014-07-13,0,0.00,0,0,0,0,0,...,0.000,0,None,None,None,None,None,None,None,2014-07
3,55,2014-08-06,2014-08-03,0,0.00,0,0,0,0,0,...,0.000,0,None,None,None,None,None,None,None,2014-08
4,55,2014-08-07,2014-08-03,0,0.00,0,0,0,0,0,...,0.000,0,None,None,None,None,None,None,None,2014-08


In [4]:
df[['total_units', 'total_sales',
       'total_orders', 'page_views', 'sessions', 'total_impressions',
       'total_clicks', 'total_spend', 'total_ad_sales',
       'new_to_brand_customers', 'CPC', 'CTR', 'CR_impressions', 'CR_clicks',
       'ROAS', 'AOV', 'CAC']] = df[['total_units', 'total_sales',
       'total_orders', 'page_views', 'sessions', 'total_impressions',
       'total_clicks', 'total_spend', 'total_ad_sales',
       'new_to_brand_customers', 'CPC', 'CTR', 'CR_impressions', 'CR_clicks',
       'ROAS', 'AOV', 'CAC']].astype('float')

In [5]:
brand_df = df[df['brandid']==brand_id]

In [6]:
brand_df = brand_df.groupby(['month']).agg({'total_units': ['sum'], 'total_sales': ['sum'], 'total_orders': ['sum'],
                                            'page_views': ['sum'], 'sessions': ['sum'], 'total_impressions': ['sum'],
                                           'total_clicks': ['sum'], 'total_spend': ['sum'], 'total_ad_sales': ['sum'],
                                           'new_to_brand_customers': ['sum'], 'CPC': ['mean'], 'CTR': ['mean'],
                                           'CR_impressions': ['mean'], 'CR_clicks': ['mean'], 'ROAS': ['mean'],
                                           'AOV': ['mean'], 'CAC': ['mean']})

In [7]:
brand_latest = brand_df[brand_df.index==latest_month].transpose()
brand_latest.columns=['Brand Latest Month']
brand_latest

,,Brand Latest Month
total_units,sum,411.000000
total_sales,sum,7915.200000
total_orders,sum,266.000000
page_views,sum,1278.000000
sessions,sum,1035.000000
total_impressions,sum,506542.000000
total_clicks,sum,1105.000000
total_spend,sum,1871.200000
total_ad_sales,sum,3588.670000
new_to_brand_customers,sum,211.000000


# Brand Year Avg Metrics

We will pull the metrics of the brand for the previous year and find the monthly average so that we can compare the brand to itself for future benchmarks.

To achieve this, we will need to first group the metrics into months, then take the average of those months across the year.

In [9]:
df['year'] = df['month'].str.slice(0, 4)
brand_year_df = df[df['year']==agg_year]
brand_year_df = brand_year_df[df['brandid']==brand_id]
brand_year_df.head()

<ipython-input-9-14f304354cdd>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  brand_year_df = brand_year_df[df['brandid']==brand_id]


,brandid,pst_date,minWeekDate,total_units,total_sales,total_orders,page_views,sessions,total_impressions,total_clicks,...,new_to_brand_customers,CPC,CTR,CR_impressions,CR_clicks,ROAS,AOV,CAC,month,year
12837,143,2021-03-30,2021-03-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03,2021
12849,143,2021-03-31,2021-03-28,0.0,0.0,0.0,3.0,3.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03,2021
12861,143,2021-04-01,2021-03-28,0.0,0.0,0.0,24.0,11.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04,2021
12873,143,2021-04-02,2021-03-28,0.0,0.0,0.0,7.0,6.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04,2021
12885,143,2021-04-03,2021-03-28,2.0,32.0,2.0,4.0,4.0,0.0,0.0,...,2.0,NaN,NaN,NaN,NaN,NaN,16.0,NaN,2021-04,2021


In [10]:
brand_year_df = brand_year_df.groupby(['month']).agg({'total_units': ['sum'], 'total_sales': ['sum'], 'total_orders': ['sum'],
                                            'page_views': ['sum'], 'sessions': ['sum'], 'total_impressions': ['sum'],
                                           'total_clicks': ['sum'], 'total_spend': ['sum'], 'total_ad_sales': ['sum'],
                                           'new_to_brand_customers': ['sum'], 'CPC': ['mean'], 'CTR': ['mean'],
                                           'CR_impressions': ['mean'], 'CR_clicks': ['mean'], 'ROAS': ['mean'],
                                           'AOV': ['mean'], 'CAC': ['mean']})

In [11]:
brand_year_df['year']=brand_year_df.index.str.slice(0, 4)
brand_year_df.head()

,total_units,total_sales,total_orders,page_views,sessions,total_impressions,total_clicks,total_spend,total_ad_sales,new_to_brand_customers,CPC,CTR,CR_impressions,CR_clicks,ROAS,AOV,CAC,year
,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,mean,mean,mean,mean,mean,mean,mean,
month,,,,,,,,,,,,,,,,,,
2021-03,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.00,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021
2021-04,435.0,6896.0,319.0,1309.0,1073.0,216025.0,408.0,765.79,2051.00,303.0,1.812240,0.002100,0.001761,0.855839,3.815846,27.442443,0.560328,2021
2021-05,1038.0,16320.0,813.0,1799.0,1473.0,176606.0,527.0,581.30,3830.01,746.0,1.234083,0.004048,0.004755,1.104071,6.754640,24.203842,0.914226,2021
2021-06,211.0,3306.4,132.0,353.0,303.0,187329.0,330.0,609.85,1232.00,99.0,1.849240,0.003257,0.001430,0.422540,2.223881,23.327179,0.175277,2021
2021-07,214.0,3408.0,115.0,365.0,317.0,132218.0,386.0,689.95,1251.01,78.0,1.777715,0.004510,0.001703,0.380235,2.113478,27.600205,0.172800,2021


In [12]:
brand_year_df = brand_year_df.groupby(['year']).mean()
brand_year_df

,total_units,total_sales,total_orders,page_views,sessions,total_impressions,total_clicks,total_spend,total_ad_sales,new_to_brand_customers,CPC,CTR,CR_impressions,CR_clicks,ROAS,AOV,CAC
,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,mean,mean,mean,mean,mean,mean,mean
year,,,,,,,,,,,,,,,,,
2021,362.9,6273.75,244.1,979.7,797.4,268408.8,552.5,1001.868,2221.827,209.2,1.750164,0.006178,0.004166,0.510025,3.033326,29.208633,0.323086


In [13]:
brand_year = brand_year_df[brand_year_df.index==agg_year].transpose()
brand_year.columns=['Brand Avg.']
brand_year

,,Brand Avg.
total_units,sum,362.900000
total_sales,sum,6273.750000
total_orders,sum,244.100000
page_views,sum,979.700000
sessions,sum,797.400000
total_impressions,sum,268408.800000
total_clicks,sum,552.500000
total_spend,sum,1001.868000
total_ad_sales,sum,2221.827000
new_to_brand_customers,sum,209.200000


# Competing Brands Avg Metrics

We will also pull the metrics of some related and competing brands for the previous year and find the monthly averages so that we can benchmark the brand against its competitors in the industry.

To achieve this, we will use multiple grouping functions. We need to first group the metrics by both brand and month. Then we can group the metrics by the averages of those months by brand for the year. Then we can average all the metrics for the brands to end up with the industry competitor's average metrics to use as bench marks.

In [14]:
comp_df = df[df['brandid']!=brand_id]

In [15]:
comp_df = comp_df.groupby(['brandid','month']).agg({'total_units': ['sum'], 'total_sales': ['sum'], 'total_orders': ['sum'],
                                            'page_views': ['sum'], 'sessions': ['sum'], 'total_impressions': ['sum'],
                                           'total_clicks': ['sum'], 'total_spend': ['sum'], 'total_ad_sales': ['sum'],
                                           'new_to_brand_customers': ['sum'], 'CPC': ['mean'], 'CTR': ['mean'],
                                           'CR_impressions': ['mean'], 'CR_clicks': ['mean'], 'ROAS': ['mean'],
                                           'AOV': ['mean'], 'CAC': ['mean']})
comp_df.head()

total_units total_sales total_orders page_views sessions  \
                        sum         sum          sum        sum      sum   
brandid month                                                              
2       2015-02         0.0         0.0          0.0        0.0      0.0   
        2015-03         0.0         0.0          0.0        0.0      0.0   
        2015-04         0.0         0.0          0.0        0.0      0.0   
        2015-05         0.0         0.0          0.0        0.0      0.0   
        2015-06         0.0         0.0          0.0        0.0      0.0   

                total_impressions total_clicks total_spend total_ad_sales  \
                              sum          sum         sum            sum   
brandid month                                                               
2       2015-02               0.0          0.0         0.0            0.0   
        2015-03               0.0          0.0         0.0            0.0   
        2015-04               0.0          0.0         0.0            0.0   
        2015-05               0.0          0.0         0.0            0.0   
        2015-06               0.0          0.0         0.0            0.0   

                new_to_brand_customers  CPC  CTR CR_impressions CR_clicks  \
                                   sum mean mean           mean      mean   
brandid month                                                               
2       2015-02                    0.0  NaN  NaN            NaN       NaN   
        2015-03                    0.0  NaN  NaN            NaN       NaN   
        2015-04                    0.0  NaN  NaN            NaN       NaN   
        2015-05                    0.0  NaN  NaN            NaN       NaN   
        2015-06                    0.0  NaN  NaN            NaN       NaN   

                ROAS  AOV  CAC  
                mean mean mean  
brandid month                   
2       2015-02  NaN  NaN  NaN  
        2015-03  NaN  NaN  NaN  
        2015-04  NaN  NaN  NaN  
        2015-05  NaN  NaN  NaN  
        2015-06  NaN  NaN  NaN

In [16]:
comp_df.reset_index(inplace=True)
comp_df['year'] = comp_df.reset_index()['month'].str.slice(0,4)
comp_df.head()

,brandid,month,total_units,total_sales,total_orders,page_views,sessions,total_impressions,total_clicks,total_spend,total_ad_sales,new_to_brand_customers,CPC,CTR,CR_impressions,CR_clicks,ROAS,AOV,CAC,year
,,,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,mean,mean,mean,mean,mean,mean,mean,
0,2,2015-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
1,2,2015-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
2,2,2015-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
3,2,2015-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
4,2,2015-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015


In [17]:
comp_df = comp_df[comp_df['year']==agg_year]
comp_df = comp_df.groupby(['brandid','year']).mean()

C:\Users\Drew\anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [18]:
comp_df = comp_df.groupby(['year']).mean()
comp_df = comp_df[comp_df.index==agg_year].transpose()
comp_df.columns=['Competition Avg.']
comp_df

,,Competition Avg.
total_units,sum,2226.083333
total_sales,sum,88195.583646
total_orders,sum,1931.541667
page_views,sum,9495.093750
sessions,sum,7551.541667
total_impressions,sum,504689.979167
total_clicks,sum,2372.645833
total_spend,sum,2746.283958
total_ad_sales,sum,24852.543229
new_to_brand_customers,sum,1130.614583


# Concat them together 

Lastly, we can concat all the data together and write it to excel for a nice and easy to read report.

In [19]:
x = brand_latest.join(brand_year)
x = x.join(comp_df)
x

,,Brand Latest Month,Brand Avg.,Competition Avg.
total_units,sum,411.000000,362.900000,2226.083333
total_sales,sum,7915.200000,6273.750000,88195.583646
total_orders,sum,266.000000,244.100000,1931.541667
page_views,sum,1278.000000,979.700000,9495.093750
sessions,sum,1035.000000,797.400000,7551.541667
total_impressions,sum,506542.000000,268408.800000,504689.979167
total_clicks,sum,1105.000000,552.500000,2372.645833
total_spend,sum,1871.200000,1001.868000,2746.283958
total_ad_sales,sum,3588.670000,2221.827000,24852.543229
new_to_brand_customers,sum,211.000000,209.200000,1130.614583


In [20]:
x.to_excel('Brand_Benchmarks.xlsx')